In [1]:
# import required packages.....
from elasticsearch import Elasticsearch, helpers
import configparser
import pandas as pd
import json
import openpyxl
from datetime import datetime
import warnings
warnings.filterwarnings("ignore") # for ignore warnings


class Elk_api:
    def __init__(self, ini_file):   #load data using cloud api keys
        self.config = configparser.ConfigParser()
        self.config.read(ini_file) #create file with extention .ini having API details
        self.es = Elasticsearch(
        cloud_id=self.config['DEFAULT']['cloud_id'],
        api_key=(self.config['DEFAULT']['apikey_id'], self.config['DEFAULT']['apikey_key']))
        
    def load_data(self, path, index): 
        split_tup = path.split('.')
        file_extension = split_tup[-1]
        if file_extension == 'csv':
            df = pd.read_csv(path)
        elif file_extension == 'json':
            df = pd.read_json(path)
        else:
            wb= openpyxl.load_workbook(path)   #Pass file and save as a workbook formate
            datestring = datetime.strftime(datetime.now(), '%b_%a_%H_%M_%p')
            dest_filename = f"{datestring}.xlsx"
            wb.save(filename = dest_filename)
            df = pd.read_excel(dest_filename)
        json_lst=list(map(lambda x : json.loads(df.iloc[x, :].to_json()), range(len(df))))
        print(f"{df.shape[0]} rows & {df.shape[1]} columns are Ready to load....")
        res=list(map(lambda data : self.es.index(index=index, document=data), json_lst))
        print(f"{len(res)} rows loaded")
        
class Elk_cloud(Elk_api):   #load data using cloud username & password
    def __init__(self, ini_file):
        self.config = configparser.ConfigParser()
        self.config.read(ini_file) #create file with extention .ini having cloud id details
        self.es = Elasticsearch(
        cloud_id=self.config['ELASTIC']['cloud_id'],
        http_auth=(self.config['ELASTIC']['user'], self.config['ELASTIC']['password']))
        
class Elk_local(Elk_api):   #load data using localhost
    def __init__(self, localhost):
        self.es = Elasticsearch(localhost)   #localhost = http://localhost:9200

In [2]:
ek=Elk_api('elkapi.ini')
ek.load_data(r"C:\elastic\elasticpy\World_data.xlsx", 'world_data_ini')

2704 rows & 27 columns are Ready to load....


ConnectionTimeout: Connection timed out